In [1]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16  # You can use a pre-trained model for feature extraction


In [2]:
image_dir = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\image_test_data"
csv_file = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\deep features_100_images (1).csv"

In [3]:
df = pd.read_csv(csv_file)
image_paths = [os.path.join(image_dir, img_name) for img_name in df['Image Name']]
income_values = df['Income'].values

In [26]:
# Define a CNN model for regression
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Flatten()) #is layer ka output lena hai

# model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # Output layer for regression

In [27]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 24, 24, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 22, 22, 128)       73856     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 7, 7, 128)        

In [5]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [6]:
X = []
for img_path in image_paths:
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.vgg16.preprocess_input(img)  # Preprocess according to the chosen model (VGG16, in this case)
    X.append(img)

In [7]:
X = np.array(X)
y = income_values

In [8]:
model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10
4/4 [==============================] - 4s 592ms/step - loss: 98240745466494976.0000
Epoch 2/10
4/4 [==============================] - 3s 569ms/step - loss: 98238537853304832.0000
Epoch 3/10
4/4 [==============================] - 2s 519ms/step - loss: 98225025886191616.0000
Epoch 4/10
4/4 [==============================] - 2s 511ms/step - loss: 98205767252836352.0000
Epoch 5/10
4/4 [==============================] - 2s 558ms/step - loss: 98146505294086144.0000
Epoch 6/10
4/4 [==============================] - 3s 618ms/step - loss: 98082604770656256.0000
Epoch 7/10
4/4 [==============================] - 3s 624ms/step - loss: 97914525520494592.0000
Epoch 8/10
4/4 [==============================] - 3s 588ms/step - loss: 97781450253795328.0000
Epoch 9/10
4/4 [==============================] - 2s 538ms/step - loss: 97551093977841664.0000
Epoch 10/10
4/4 [==============================] - 2s 514ms/step - loss: 97230826856513536.0000


In [9]:
model.save('income_cnn_model.h5')

c:\Users\Kanishk Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
feature_extractor_model = Model(inputs=model.input, outputs=model.layers[-2].output)


In [11]:
deep_features = feature_extractor_model.predict(X)

4/4 [==============================] - 1s 134ms/step


In [12]:
deep_features_df = pd.DataFrame(deep_features)

In [13]:
deep_features_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,0.0,2972567.500,0.0,2987330.25,0.0,0.0,0.0,0.0,...,0.0,2980728.00,0.0,0.0,2969862.75,2988124.50,2987099.000,2989985.50,0.0,2987033.25
1,0.0,0.0,0.0,3815630.500,0.0,3834673.50,0.0,0.0,0.0,0.0,...,0.0,3827202.75,0.0,0.0,3812509.00,3835433.00,3834604.500,3838239.00,0.0,3834285.50
2,0.0,0.0,0.0,2429789.500,0.0,2441506.75,0.0,0.0,0.0,0.0,...,0.0,2436373.50,0.0,0.0,2427640.50,2442025.75,2441212.250,2444120.00,0.0,2441087.50
3,0.0,0.0,0.0,3691685.500,0.0,3710297.00,0.0,0.0,0.0,0.0,...,0.0,3701852.00,0.0,0.0,3688568.75,3710965.50,3710712.250,3713400.50,0.0,3710223.00
4,0.0,0.0,0.0,3918409.500,0.0,3937789.00,0.0,0.0,0.0,0.0,...,0.0,3929863.50,0.0,0.0,3915170.50,3939188.00,3938528.750,3941753.50,0.0,3937919.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0,0.0,0.0,3985474.000,0.0,4004667.75,0.0,0.0,0.0,0.0,...,0.0,3996357.50,0.0,0.0,3981518.50,4005916.75,4005000.500,4008435.25,0.0,4004813.50
94,0.0,0.0,0.0,4071181.500,0.0,4090478.75,0.0,0.0,0.0,0.0,...,0.0,4082877.50,0.0,0.0,4065912.00,4092259.50,4091493.750,4093614.00,0.0,4090855.00
95,0.0,0.0,0.0,1678502.125,0.0,1686751.75,0.0,0.0,0.0,0.0,...,0.0,1683593.50,0.0,0.0,1676652.75,1686885.00,1686801.625,1687571.25,0.0,1686432.75
96,0.0,0.0,0.0,3946294.500,0.0,3966056.25,0.0,0.0,0.0,0.0,...,0.0,3957281.00,0.0,0.0,3942741.50,3967208.00,3966402.500,3969963.00,0.0,3965951.25
